In [ ]:
import streamlit as st
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# 🎨 Page Config
st.set_page_config(
    page_title="PREDICTIFY - The IPL Win Forecaster 🏆",
    page_icon="🏏",
    layout="centered",
)

# 🌈 Custom CSS Styling
st.markdown("""
    <style>
    .main {
        background: linear-gradient(135deg, #001F3F, #0074D9);
        color: white;
    }
    h1, h2, h3 {
        color: #FFD700;
        text-align: center;
        font-family: 'Trebuchet MS', sans-serif;
    }
    .footer {
        text-align: center;
        color: #FFD700;
        margin-top: 50px;
        font-size: 16px;
    }
    </style>
""", unsafe_allow_html=True)

# 🏏 Title
st.title("🏏 PREDICTIFY - The IPL Win Forecaster")
st.markdown("### Predict which team will win today's match ⚡")

# 🧠 Load trained model
with open("pipe (1).pkl", "rb") as f:
    pipe = pickle.load(f)

# 🏟️ Teams & Cities
team = [
    'Kings XI Punjab', 'Mumbai Indians', 'Kolkata Knight Riders',
    'Rajasthan Royals', 'Chennai Super Kings', 'Sunrisers Hyderabad',
    'Delhi Capitals', 'Lucknow Super Giants', 'Royal Challengers Bengaluru'
]
cities = [
    'Bangalore', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
    'Hyderabad', 'Chennai', 'Cape Town', 'Port Elizabeth', 'Durban',
    'Centurion', 'East London', 'Johannesburg', 'Kimberley',
    'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
    'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi', 'Abu Dhabi', 'Bengaluru',
    'Indore', 'Dubai', 'Sharjah', 'Navi Mumbai', 'Lucknow', 'Guwahati', 'Mohali'
]

# 🧩 Input Section
col1, col2 = st.columns(2)
with col1:
    batting_team = st.selectbox('🏏 Select Batting Team', team)
with col2:
    bowling_team = st.selectbox('🎯 Select Bowling Team', team)

selected_city = st.selectbox('🌆 Select Match City', sorted(cities))
target = st.number_input('🎯 Target Score', min_value=0)

col3, col4, col5 = st.columns(3)
with col3:
    score = st.number_input('💥 Current Score', min_value=0)
with col4:
    overs = st.number_input('⏱ Overs Completed', min_value=0, max_value=20)
with col5:
    wickets = st.number_input('🚨 Wickets Down', min_value=0, max_value=10)

# 🧮 Prediction
if st.button('🔮 Predict Probability'):
    if overs == 0:
        st.warning("⚠️ Overs cannot be zero.")
    else:
        runs_left = target - score
        balls_left = 120 - (overs * 6)
        wickets_left = 10 - wickets
        crr = score / overs
        rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0

        input_df = pd.DataFrame({
            'batting_team': [batting_team],
            'bowling_team': [bowling_team],
            'city': [selected_city],
            'runs_left': [runs_left],
            'balls_left': [balls_left],
            'wickets_left': [wickets_left],
            'total_runs_x': [target],
            'crr': [crr],
            'rrr': [rrr]
        })

        result = pipe.predict_proba(input_df)
        loss = result[0][0]
        win = result[0][1]

        # 🏁 Display Results
        st.subheader("📊 Win Probability:")
        st.markdown(
            f"""
            <h2 style='color:#00FFAA;text-align:center;'>{batting_team}: {round(win*100)}%</h2>
            <h2 style='color:#FF6B6B;text-align:center;'>{bowling_team}: {round(loss*100)}%</h2>
            """,
            unsafe_allow_html=True
        )

        # Prepare Data for Charts
        prob_df = pd.DataFrame({
            'Team': [batting_team, bowling_team],
            'Win Probability (%)': [win * 100, loss * 100]
        })

        # 🍩 Donut Chart (Top)
        fig_donut = go.Figure(data=[go.Pie(
            labels=[batting_team, bowling_team],
            values=[win * 100, loss * 100],
            hole=0.5,
            marker=dict(colors=['#00FFAA', '#FF6B6B']),
            textinfo='label+percent',
            textfont=dict(size=18, color='white')
        )])
        fig_donut.update_layout(
            showlegend=False,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            height=400
        )
        st.plotly_chart(fig_donut, use_container_width=True)

        # 📊 Bar Chart (Below)
        fig_bar = px.bar(
            prob_df,
            x='Team',
            y='Win Probability (%)',
            color='Team',
            text='Win Probability (%)',
            color_discrete_sequence=['#00FFAA', '#FF6B6B']
        )
        fig_bar.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
        fig_bar.update_layout(
            yaxis=dict(range=[0, 100]),
            xaxis_title="Name of team",
            yaxis_title="Probability",
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            showlegend=False,
            height=400
        )
        st.plotly_chart(fig_bar, use_container_width=True)

# ✨ Footer
st.markdown("<div class='footer'>Made with ❤️ by TechTitan's Team | IPL Predictor</div>", unsafe_allow_html=True)

